In [3]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import torch
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import shutil
# Định nghĩa CustomDataset
from utils import *
import gc
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.utils import resample

def unique_rows(matrix):
    seen = set()
    unique_matrix = []
    for row in matrix:
        row_tuple = tuple(row)
        if row_tuple not in seen:
            seen.add(row_tuple)
            unique_matrix.append(row)
    return np.vstack(unique_matrix)

def unique_data(data_t, labels_t):
    data_flat = data_t.reshape(data_t.shape[0], -1)
    datala_t = np.hstack((data_flat, labels_t.reshape(-1, 1)))
    datala_t = unique_rows(datala_t)
    data = datala_t[:, :-1].reshape(-1, data_t.shape[1], 9).astype(np.float32)
    labels = datala_t[:, -1].astype(int)
    return data, labels

batch_size = 2048
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for time_window in [30]:
    for h_windows in [32, 16, 8]:
        print(f'{str(time_window)}_{str(h_windows)}')
        with h5py.File('/mnt/d/dataset_pre/CICDDoS2017/'+str(time_window)+str(h_windows)+'_data_fix.h5', 'r') as hf:
            data_cic = hf['data'][:]
        with h5py.File('/mnt/d/dataset_pre/CICDDoS2017/'+str(time_window)+str(h_windows)+'_labels_fix.h5', 'r') as hf:
            labels_cic = hf['labels'][:]
        data_cic, labels_cic = unique_data(data_cic, labels_cic)
        with h5py.File('/mnt/e/botiot/'+str(time_window)+str(h_windows)+'_data.h5', 'r') as hf:
            data = hf['data'][:]
        with h5py.File('/mnt/e/botiot/'+str(time_window)+str(h_windows)+'_labels.h5', 'r') as hf:
            labels_ = hf['labels'][:]
            labels_[np.where(labels_ == 2)] = 1
        n_samples, height, width = data.shape
        train_data_reshaped = data.reshape(n_samples, height * width)
        # Áp dụng SMOTE
        sm = SMOTE(random_state=42)
        train_shape = train_data_reshaped.shape[0]
        train_data_res, train_labels_res = sm.fit_resample(train_data_reshaped, labels_)
        train_data_res1, train_labels_res1 = sm.fit_resample(train_data_reshaped, labels_)
        train_data_res = np.vstack([train_data_res,train_data_res1])
        train_labels_res = np.hstack([train_labels_res,train_labels_res1])
        train_data_res1, train_labels_res1 = sm.fit_resample(train_data_reshaped, labels_)
        train_data_res = np.vstack([train_data_res,train_data_res1])
        train_labels_res = np.hstack([train_labels_res,train_labels_res1])
        del train_data_res1
        del train_labels_res1
        gc.collect()
        data = train_data_res
        labels_ = train_labels_res
        del train_data_res
        del train_labels_res
        gc.collect() 
        data.reshape(data.shape)
        data = data.reshape(-1,h_windows,9)
        data = np.concatenate([data,data_cic])
        labels_ = np.concatenate([labels_,labels_cic])
        del data_cic
        del labels_cic
        gc.collect()
        data_reshaped = data.reshape(-1, data.shape[-1])
        scaler = MinMaxScaler()
        normalized_data_reshaped = scaler.fit_transform(data_reshaped)
        data = normalized_data_reshaped.reshape(data.shape)
        class CustomDataset(Dataset):
            def __init__(self, data, labels):
                self.data = data
                self.labels = labels

            def __len__(self):
                return len(self.data)

            def __getitem__(self, idx):
                sample = self.data[idx]
                label = self.labels[idx]
                sample = torch.tensor(sample, dtype=torch.float32).unsqueeze(0)  # Thêm chiều mới tại vị trí index 0
                label = torch.tensor(label, dtype=torch.long)
                return sample, label
        dataset = CustomDataset(data, labels_)
        train_idx, temp_idx, train_labels, temp_labels = train_test_split(
            np.arange(len(data)), labels_, test_size=0.4, stratify=labels_, random_state=42)
        valid_idx, test_idx, valid_labels, test_labels = train_test_split(
            temp_idx, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)
        train_data = data[train_idx]
        train_labels = labels_[train_idx]
        train_dataset_resampled = CustomDataset(train_data, train_labels)

        valid_dataset = Subset(dataset, valid_idx)
        test_dataset = Subset(dataset, test_idx)
        train_loader_resampled = DataLoader(train_dataset_resampled, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        for batch_data, batch_labels in train_loader_resampled:
            print('Train batch after SMOTE:', batch_data.shape, batch_labels.shape)
            break  # chỉ in ra một batch để kiểm tra

        for batch_data, batch_labels in valid_loader:
            print('Validation batch:', batch_data.shape, batch_labels.shape)
            break 

        for batch_data, batch_labels in test_loader:
            print('Test batch:', batch_data.shape, batch_labels.shape)
            break  
        for quant in ['FP', 'TNN', 'TBN', 'BNN']:
            if quant == 'FP':
                class Q_CAD(nn.Module):
                    def __init__(self):
                        super(Q_CAD, self).__init__()
                        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3,3), padding=1)
                        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), padding=1)
                        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                        self.fc1 = nn.Linear(in_features=64 * int(h_windows/4) * 2, out_features=128)
                        self.fc2 = nn.Linear(in_features=128, out_features=2) 
                        
                    def forward(self, x):
                        x = F.relu(self.pool(self.conv1(x)))
                        x = F.relu(self.pool(self.conv2(x)))
                        x = x.view(x.size(0), -1)
                        x = F.relu(self.fc1(x))
                        x = self.fc2(x)
                        return x
            else:
                class Q_CAD(nn.Module):
                    def __init__(self):
                        super(Q_CAD, self).__init__()
                        self.conv1 = QConv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
                        self.conv2 = QConv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, quant=quant)
                        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                        self.fc1 = QLinear(in_features=64 * int(h_windows/4) * 2, out_features=128, quant=quant)
                        self.fc2 = QLinear(in_features=128, out_features=2, quant=quant)
                        
                    def forward(self, x):
                        x = self.pool(self.conv1(x))
                        x = self.pool(self.conv2(x))
                        x = x.view(x.size(0), -1)
                        x = self.fc1(x)
                        x = self.fc2(x)
                        return x

            model_fp = Q_CAD().to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model_fp.parameters(), lr=0.001)
            best_accuracy = 0.0
            best_epoch = 0
            num_epochs = 2
            for epoch in range(num_epochs):
                model_fp.train()
                for i, (images, labels) in enumerate(train_loader_resampled):
                    images = images.to(device)
                    labels = labels.to(device)
                    
                    outputs = model_fp(images)
                    loss = criterion(outputs, labels)
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    if (i+1) % 100 == 0:
                        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader_resampled)}], Loss: {loss.item():.4f}', end='\r')
                
                model_fp.eval()
                with torch.no_grad():
                    correct = 0
                    total = 0
                    all_labels = []
                    all_predictions = []
                    for images, labels in valid_loader:
                        images = images.to(device)
                        labels = labels.to(device)
                        
                        outputs = model_fp(images)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        all_labels.extend(labels.cpu().numpy())
                        all_predictions.extend(predicted.cpu().numpy())

                    accuracy = 100 * correct / total
                    f1 = f1_score(all_labels, all_predictions, average='weighted')
                    precision = precision_score(all_labels, all_predictions, average='weighted')
                    recall = recall_score(all_labels, all_predictions, average='weighted')
                    cm = confusion_matrix(all_labels, all_predictions)
                    tn, fp, fn, tp = cm.ravel()
                    fpr = fp / (fp + tn)
                    ppv = precision
                    tpr = recall

                    print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%, F1: {f1:.2f}, PPV: {ppv:.2f}, TPR: {tpr:.2f}, FPR: {fpr:.4f}')

                    # Lưu lại mô hình nếu độ chính xác tốt hơn
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_epoch = epoch + 1
                        torch.save(model_fp.state_dict(), 'best_model.pth')

            print(f'Best Validation Accuracy: {best_accuracy:.2f}% at epoch {best_epoch}')

            # Đánh giá cuối cùng trên tập test
            model_fp.load_state_dict(torch.load('best_model.pth'))
            model_fp.eval()
            correct = 0
            total = 0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    
                    outputs = model_fp(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())

            test_accuracy = 100 * correct / total
            f1 = f1_score(all_labels, all_predictions, average='weighted')
            precision = precision_score(all_labels, all_predictions, average='weighted')
            recall = recall_score(all_labels, all_predictions, average='weighted')
            cm = confusion_matrix(all_labels, all_predictions)
            tn, fp, fn, tp = cm.ravel()
            fpr = fp / (fp + tn)
            ppv = precision
            tpr = recall

            print(f'Test Accuracy: {test_accuracy:.2f}%, F1: {f1:.2f}, PPV: {ppv:.2f}, TPR: {tpr:.2f}, FPR: {fpr:.4f}')
            shutil.copy('best_model.pth', f'model_save/{quant}{str(time_window)}{str(h_windows)}.pth')

            file_path = '/home/usami/TBN64/train/model_acc.txt'
            new_line = f'{quant}{str(time_window)}{str(h_windows)}: {test_accuracy:.2f}%, F1: {f1:.2f}, PPV: {ppv:.2f}, TPR: {tpr:.2f}, FPR: {fpr:.4f}\n'
            with open(file_path, 'a') as file:
                file.write(new_line)
        gc.collect()
        with open(file_path, 'a') as file:
            file.write('\n')

30_32
Train batch after SMOTE: torch.Size([2048, 1, 32, 9]) torch.Size([2048])
Validation batch: torch.Size([2048, 1, 32, 9]) torch.Size([2048])
Test batch: torch.Size([2048, 1, 32, 9]) torch.Size([2048])


KeyboardInterrupt: 

In [7]:
images[0]

tensor([[[3.5294e-01, 8.4270e-04, 1.0000e+00, 1.0720e-01, 5.7831e-05,
          2.1927e-03, 4.3373e-05, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.000